In [13]:
example1 = """
Monkey 0:
  Starting items: 79, 98
  Operation: new = old * 19
  Test: divisible by 23
    If true: throw to monkey 2
    If false: throw to monkey 3

Monkey 1:
  Starting items: 54, 65, 75, 74
  Operation: new = old + 6
  Test: divisible by 19
    If true: throw to monkey 2
    If false: throw to monkey 0

Monkey 2:
  Starting items: 79, 60, 97
  Operation: new = old * old
  Test: divisible by 13
    If true: throw to monkey 1
    If false: throw to monkey 3

Monkey 3:
  Starting items: 74
  Operation: new = old + 3
  Test: divisible by 17
    If true: throw to monkey 0
    If false: throw to monkey 1
"""

In [14]:
data = open("input.txt").read()

In [15]:
import re, math
from pprint import pprint
from dataclasses import dataclass, field


@dataclass
class Monkey:
    operation: str
    test: str
    targets: list[int]  # False, True monkey targets
    inspect_counter: int = 0
    items: list[int] = field(default_factory=list)

    def execute_operation(self, value: int) -> int:
        if not re.match(r"new = old [-+*/] (\d+|old)", self.operation):
            raise ValueError(f"bad operation {self.operation}")
        old = value
        return eval(self.operation.split("=")[1])

    def execute_test(self, value: int) -> int:
        divider = int(self.test.split()[-1])
        ret = not (value % divider)
        return self.targets[ret]

    def inspect(self, value: int):
        self.inspect_counter = self.inspect_counter + 1
        return self.execute_operation(value)


def parse(data):
    parse_re = re.compile(
        """
    Monkey\s(?P<monkey>\d+):
    \s+Starting\sitems:\s*(?P<items>[\d+, ]+)
    \s*Operation:\s*(?P<op>.+)
    \s*Test:\s*(?P<test>.+)
    \s*If\strue:.+\s(?P<target_true>\d+)
    \s*If\sfalse:.+\s(?P<target_false>\d+)
    """,
        flags=re.M | re.X,
    )
    monkeys = []
    for m in parse_re.finditer(data):
        monkey = Monkey(
            operation=m.group("op"),
            test=m.group("test"),
            targets=[int(e) for e in (m.group("target_false"), m.group("target_true"))],
            items=[int(e) for e in m.group("items").split(", ")],
        )
        monkeys.append(monkey)
    return monkeys


def execute_round(monkeys, bored=3, reduce_f=lambda x: x):
    for m in monkeys:
        for item in m.items:
            item = m.inspect(item)
            item = item // bored
            item = reduce_f(item)
            monkeys[m.execute_test(item)].items.append(item)
        m.items.clear()
    return monkeys


def task1(data):
    monkeys = parse(data)
    pprint(monkeys)
    for r in range(20):
        execute_round(monkeys)

    monkeys_counters = sorted(m.inspect_counter for m in monkeys)
    return monkeys_counters[-1] * monkeys_counters[-2]


print(task1(data))

[Monkey(operation='new = old * 13',
        test='divisible by 5',
        targets=[6, 1],
        inspect_counter=0,
        items=[52, 78, 79, 63, 51, 94]),
 Monkey(operation='new = old + 3',
        test='divisible by 7',
        targets=[3, 5],
        inspect_counter=0,
        items=[77, 94, 70, 83, 53]),
 Monkey(operation='new = old * old',
        test='divisible by 13',
        targets=[6, 0],
        inspect_counter=0,
        items=[98, 50, 76]),
 Monkey(operation='new = old + 5',
        test='divisible by 11',
        targets=[7, 5],
        inspect_counter=0,
        items=[92, 91, 61, 75, 99, 63, 84, 69]),
 Monkey(operation='new = old + 7',
        test='divisible by 3',
        targets=[0, 2],
        inspect_counter=0,
        items=[51, 53, 83, 52]),
 Monkey(operation='new = old + 4',
        test='divisible by 2',
        targets=[7, 4],
        inspect_counter=0,
        items=[76, 76]),
 Monkey(operation='new = old * 19',
        test='divisible by 17',
        tar

In [16]:
def task2(data):
    monkeys = parse(data)
    pprint(monkeys)
    monkey_lcm = math.lcm(*[int(m.test.split()[-1]) for m in monkeys])

    def reduce_(v):
        v %= monkey_lcm
        return v

    for r in range(10000):
        if r % 1000 == 0:
            print(r)
        execute_round(monkeys, bored=1, reduce_f=reduce_)

    monkeys_counters = sorted(m.inspect_counter for m in monkeys)
    return monkeys_counters[-1] * monkeys_counters[-2]


task2(example1)

[Monkey(operation='new = old * 19',
        test='divisible by 23',
        targets=[3, 2],
        inspect_counter=0,
        items=[79, 98]),
 Monkey(operation='new = old + 6',
        test='divisible by 19',
        targets=[0, 2],
        inspect_counter=0,
        items=[54, 65, 75, 74]),
 Monkey(operation='new = old * old',
        test='divisible by 13',
        targets=[3, 1],
        inspect_counter=0,
        items=[79, 60, 97]),
 Monkey(operation='new = old + 3',
        test='divisible by 17',
        targets=[1, 0],
        inspect_counter=0,
        items=[74])]
0
1000
2000
3000
4000
5000
6000
7000
8000
9000


2713310158

In [17]:
task2(data)

[Monkey(operation='new = old * 13',
        test='divisible by 5',
        targets=[6, 1],
        inspect_counter=0,
        items=[52, 78, 79, 63, 51, 94]),
 Monkey(operation='new = old + 3',
        test='divisible by 7',
        targets=[3, 5],
        inspect_counter=0,
        items=[77, 94, 70, 83, 53]),
 Monkey(operation='new = old * old',
        test='divisible by 13',
        targets=[6, 0],
        inspect_counter=0,
        items=[98, 50, 76]),
 Monkey(operation='new = old + 5',
        test='divisible by 11',
        targets=[7, 5],
        inspect_counter=0,
        items=[92, 91, 61, 75, 99, 63, 84, 69]),
 Monkey(operation='new = old + 7',
        test='divisible by 3',
        targets=[0, 2],
        inspect_counter=0,
        items=[51, 53, 83, 52]),
 Monkey(operation='new = old + 4',
        test='divisible by 2',
        targets=[7, 4],
        inspect_counter=0,
        items=[76, 76]),
 Monkey(operation='new = old * 19',
        test='divisible by 17',
        tar

14952185856

In [18]:
14548077720

14548077720